In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from IPython.display import Markdown as md

In [2]:
df = pd.read_pickle("shared/Motor_Vehicle_Collisions_-_Crashes.pkl")
df


,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2021-09-11 02:39:00,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,2.0,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,2022-03-26 11:45:00,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,2022-06-29 06:55:00,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,2021-09-11 09:35:00,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,0.0,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,2021-12-14 08:13:00,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018240,2023-07-03 18:05:00,NaN,NaN,40.866806,-73.931010,"(40.866806, -73.93101)",RIVERSIDE DRIVE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4648110,Sedan,Sedan,NaN,NaN,NaN
2018241,2023-07-22 21:39:00,BRONX,10457.0,40.844177,-73.902920,"(40.844177, -73.90292)",EAST 174 STREET,WEBSTER AVENUE,NaN,1.0,...,NaN,NaN,NaN,NaN,4648117,Sedan,NaN,NaN,NaN,NaN
2018242,2023-07-02 17:55:00,MANHATTAN,10006.0,40.711033,-74.014540,"(40.711033, -74.01454)",WEST STREET,LIBERTY STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4648366,Taxi,NaN,NaN,NaN,NaN
2018243,2023-07-22 13:15:00,QUEENS,11433.0,40.691580,-73.793190,"(40.69158, -73.79319)",110 AVENUE,157 STREET,NaN,1.0,...,Driver Inattention/Distraction,NaN,NaN,NaN,4648129,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN


In [3]:
#Borough with the maximum number of crashes reported since 2012
df["BOROUGH"].value_counts()

BOROUGH
BROOKLYN         441026
QUEENS           372457
MANHATTAN        313266
BRONX            205345
STATEN ISLAND     58297
Name: count, dtype: int64

In [4]:
#Borough with the maximum number of crashes adjusted for population
pop = pd.DataFrame(data = {'population': [1446788, 2648452, 1638281, 2330295, 487155], 
             'BOROUGH': ['BRONX', 'BROOKLYN', 'MANHATTAN', 'QUEENS', 'STATEN ISLAND']},)
pop

,population,BOROUGH
0,1446788,BRONX
1,2648452,BROOKLYN
2,1638281,MANHATTAN
3,2330295,QUEENS
4,487155,STATEN ISLAND


In [5]:
crash_count = df.groupby('BOROUGH').size().reset_index(name='crash_count')
crash_count

,BOROUGH,crash_count
0,BRONX,205345
1,BROOKLYN,441026
2,MANHATTAN,313266
3,QUEENS,372457
4,STATEN ISLAND,58297


In [6]:
merged_df = pop.merge(crash_count, on = "BOROUGH", how = "left")
merged_df

,population,BOROUGH,crash_count
0,1446788,BRONX,205345
1,2648452,BROOKLYN,441026
2,1638281,MANHATTAN,313266
3,2330295,QUEENS,372457
4,487155,STATEN ISLAND,58297


In [7]:
merged_df['crash_per_100000'] = (merged_df['crash_count'] / merged_df['population']) * 100000
merged_df.sort_values(["crash_per_100000"],ascending = False)

,population,BOROUGH,crash_count,crash_per_100000
2,1638281,MANHATTAN,313266,19121.628097
1,2648452,BROOKLYN,441026,16652.217975
3,2330295,QUEENS,372457,15983.255339
0,1446788,BRONX,205345,14193.164444
4,487155,STATEN ISLAND,58297,11966.827806


In [8]:
#Analyzing the leading cause of crashes
vehicle1 = df.drop(df[df["CONTRIBUTING FACTOR VEHICLE 1"] == "Unspecified"].index)
vehicle1

,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2021-09-11 02:39:00,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,2.0,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,2022-03-26 11:45:00,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,2022-06-29 06:55:00,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
4,2021-12-14 08:13:00,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN
6,2021-12-14 17:05:00,NaN,NaN,40.709183,-73.956825,"(40.709183, -73.956825)",BROOKLYN QUEENS EXPRESSWAY,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4486555,Sedan,Tractor Truck Diesel,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018238,2023-07-22 10:40:00,NaN,NaN,NaN,NaN,NaN,CLEARVIEW EXPRESSWAY,NORTHERN BOULEVARD,NaN,3.0,...,Unspecified,NaN,NaN,NaN,4647804,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN
2018239,2023-06-16 00:00:00,NaN,NaN,40.854310,-73.930090,"(40.85431, -73.93009)",WEST 189 STREET,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,4648255,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
2018240,2023-07-03 18:05:00,NaN,NaN,40.866806,-73.931010,"(40.866806, -73.93101)",RIVERSIDE DRIVE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4648110,Sedan,Sedan,NaN,NaN,NaN
2018242,2023-07-02 17:55:00,MANHATTAN,10006.0,40.711033,-74.014540,"(40.711033, -74.01454)",WEST STREET,LIBERTY STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4648366,Taxi,NaN,NaN,NaN,NaN


In [9]:
vehicle1["CONTRIBUTING FACTOR VEHICLE 1"].value_counts(normalize = True).head(3)

CONTRIBUTING FACTOR VEHICLE 1
Driver Inattention/Distraction    0.304180
Failure to Yield Right-of-Way     0.090335
Following Too Closely             0.081466
Name: proportion, dtype: float64

In [10]:
#Top 3 causes of crashes
vehicle1["CONTRIBUTING FACTOR VEHICLE 1"].value_counts(normalize = True).head(3).sum()

0.47598056643578757

In [11]:
#Analyzing fatalities
len(df[df["NUMBER OF PERSONS KILLED"] >= 1])

2804

In [12]:
#Likelihood of fatal accidents
len(df[df['NUMBER OF PERSONS KILLED'] >= 1])/len(df)*1000

1.3893258747079764

In [13]:
#Missing data
df["BOROUGH"].isnull().sum()/len(df)

0.3110890897784957

In [14]:
#Which combinations of vehicles have the most number of accidents
df['Vehicle Combination'] = df['VEHICLE TYPE CODE 1'] + ' / ' + df['VEHICLE TYPE CODE 2']
df['Vehicle Combination'].value_counts(ascending = False).head(1)

Vehicle Combination
Sedan / Sedan    197944
Name: count, dtype: int64